# Haw River Levels
Data feed from [USGS REST API](https://waterservices.usgs.gov/rest/IV-Service.html)

In [ ]:
import ipyleaflet as lf
import ipywidgets as widgets
import pandas as pd
import requests

In [ ]:
url = 'https://waterservices.usgs.gov/nwis/iv/?format=json' + \
        '&sites=02096960,02096500' + \
        '&startDT=2018-09-17' + \
        '&endDT=2018-09-19' + \
        '&parameterCd=00060,00065' + \
        '&siteStatus=all'

def get_location_data(location):
    '''Extracts location data from a timeSeries data array.
    Returns a dictionary of site data and a dataframe of
    the metric for that location.'''
    
    # Time series array to time series dataframe
    metrics = pd.DataFrame.from_records(location['values'][0]['value'])
    
    # Set data types
    metrics['value'] = metrics['value'].astype('float64')
    metrics['dateTime'] = metrics['dateTime'].astype('datetime64')
    
    # Add time limits to meta data
    start = metrics['dateTime'].min()
    end = metrics['dateTime'].max()
    
    # Set value name to metric description
    description = location['variable']['variableDescription']
    description = description.replace(' ','_').replace(',','').lower()
    metrics.rename(columns={'value':description}, inplace=True)
    
    # Set index to timescale
    metrics.set_index('dateTime', inplace=True)
    
    # Change array in qualifiers to string
    if type(metrics.qualifiers[0]) == list:
        
        metrics['status'] = [i[0] for i in metrics.qualifiers]
    
    # Dictionary of metadata with metrics dataframe
    source = location['sourceInfo']
    geo_info = source['geoLocation']['geogLocation']
    
    site_data = {
        'site_name':source['siteName'],
        'site_code':source['siteCode'][0]['value'],
        'network':source['siteCode'][0]['network'],
        'projection':geo_info['srs'],
        'latitude':geo_info['latitude'],
        'longitude':geo_info['longitude'],
        'coordinates':(geo_info['latitude'],
                       geo_info['longitude']),
        'measurement':location['variable']['variableName'],
        'description':description,
        'start_datetime': start,
        'end_datetime': end,
        'metrics':metrics,
                }
    
    
    return (site_data)



In [ ]:
# Future Main Loop

data = requests.get(url)
raw_data = data.json()
time_series = raw_data['value']['timeSeries']

dataset = {}
for location in time_series:
    site_data = get_location_data(location) 
    print(site_data['metrics'])
